In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from find_screen_utils import *
from find_object_utils import *
from __future__ import division

#Extract Screen From Gaze Video

In [2]:
infile = 'images/tobii/border/video/video_disp_gaze.m4v'
outfile = 'images/tobii/border/video/video_disp_gaze_screen2.m4v'

In [1]:
vid = cv2.VideoCapture(infile)

size = (1280, 720)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS), size, True)

last_good = np.zeros((720, 1280, 3), dtype=np.uint8)  # represents the last good frame

i = 1
tot = vid.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        proc_frame = process_frame_lines(frame, 'light')
        
        if proc_frame is None:  # if screen isn't found, save image of frame
            out.write(last_good)  # write the last good frame to the video
            cv2.imwrite('images/tobii/border/video/dropped/frame%i.bmp'%(i), frame)
            pass
        else:  # frame is good
            last_good = proc_frame
            out.write(proc_frame)
        i += 1
    else:
        break


vid.release()
out.release()

#Plot Gaze Onto Still Image With Object Detection

In [2]:
gaze = pd.read_csv('images/gallery/sonhouse.csv')

gaze = gaze[~gaze.vts_time.isnull()] # only start tracking eyes once video starts

infile = 'images/gallery/sonhouse.mp4'
outfile = 'images/gallery/sonhouse_org_gaze.m4v'

In [3]:
vid = cv2.VideoCapture(infile)
match = cv2.imread('images/gallery/sonhouse.jpg')
gray_match = cv2.cvtColor(match, cv2.COLOR_BGR2GRAY)
sift = cv2.SIFT()
match_sift = sift.detectAndCompute(gray_match, None)

size = (match.shape[1], match.shape[0])
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS)*2, size, True)
# note doubled framerate to include all eye tracking coordinates

i = 1
tot = vid.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)*2

gaze_val = gaze['gaze_pos_val'].values
gaze_x = gaze['gaze_pos_x'].values
gaze_y = gaze['gaze_pos_y'].values
vts = gaze['vts_time'].values / 1000.

while(vid.isOpened()):
    vid_time = vid.get(cv2.cv.CV_CAP_PROP_POS_MSEC)
    ret, frame = vid.read()
    if ret:
        match_cp = match.copy()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        M = object_find(match_sift, gray_frame, 10)
        while vts[i] < vid_time:
            if gaze_val[i] == 0 and M is not None:
                org_pos = np.array((1920*gaze_x[i], 1080*gaze_y[i])).reshape(-1,1,2)
                trans_pos = cv2.perspectiveTransform(org_pos, M)
                trans_pos = tuple(np.int32(trans_pos[0,0]))
                if (trans_pos[0] <= size[0] and trans_pos[0] >= 0 and
                    trans_pos[1] <= size[1] and trans_pos[1] >= 0):
                    cv2.circle(match_cp, trans_pos, 8, [255,0,0], -2) # draw blue circle on current frame
                    cv2.circle(match, trans_pos, 8, [0,255,0], 2) # draw green circle as trace
            out.write(match_cp)
            
            i += 1
        if i%50 == 0:
            print '%0.2f%%'%((i/tot)*100)
    else:
        break


vid.release()
out.release()

1.43%
15.76%
17.19%
18.62%
20.06%
21.49%
22.92%
24.36%
25.79%
27.22%
28.65%
30.09%
31.52%
32.95%
34.38%
35.82%
37.25%
38.68%
40.11%
41.55%
42.98%
44.41%
45.85%
47.28%
48.71%
50.14%
51.58%
53.01%
54.44%
55.87%
57.31%
58.74%
83.09%
84.53%
85.96%
87.39%
88.83%
90.26%
91.69%
93.12%
94.56%
95.99%
97.42%
